## Python + SQl

In [1]:
password = input("Introduzca su contraseña de usuario Root")

In [69]:
import mysql.connector
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# realizando una conexión a una bases de datos en MySQL

connect = mysql.connector.connect(host="localhost",                                             
                                   user="root",
                                   password=password,
                                   port="3306",
                                   database="northwind",
                                   auth_plugin="mysql_native_password")  

#### Structured Query Language
Obteniendo la informacón del almacén informático mas organizado, las bases de datos relacionales, de forma resumida son un cúmulo de datos pertenecientes a un mismo contexto y organizados sistemáticamente para su posterior uso y cuya manera de interactuar con el servidor es mediante SQL

In [ ]:
first_queries = connect.cursor()

first_queries.execute('''create view order_details_cash as
                        select OrderID, od.ProductID, Quantity, Price * Quantity as product_cash from OrderDetails od
                        join Products p on od.ProductID = p.ProductID;''')

first_queries.execute('''create view products_revenue as
                        select p.ProductID, ProductName, sum(product_cash) as product_revenue, sum(Quantity) from order_details_cash odc
                        join Products p on p.ProductID = odc.ProductID
                        group by p.ProductID;''')

first_queries.execute('''select * from products_revenue where product_revenue > (select avg(product_revenue) from products_revenue)
                        order by product_revenue desc''')

df_products = pd.DataFrame(first_queries.fetchall())
df_products.columns = ["product_id","product","product_revenue","total_quantity"]

mean_products = df_products["product_revenue"].mean()

df_products

##### Primer Data Frame constituido por los productos vendidos, su cantidad e ingresos

In [ ]:
second_queries = connect.cursor()

second_queries.execute('''create view orders_revenue as
                        select o.OrderID, sum(product_cash) as order_revenue, sum(Quantity) as total_quantity, EmployeeID from order_details_cash odc
                        join Orders o on o.OrderID = odc.OrderID
                        group by o.OrderID
                        order by order_revenue desc;''')

second_queries.execute('''select * from orders_revenue where order_revenue > (select avg(order_revenue) from orders_revenue)''')

df_orders = pd.DataFrame(second_queries.fetchall())
df_orders.drop(3,axis=1,inplace=True)
df_orders.columns = ["order_id","order_revenue","total_quantity"]

df_orders["order_id"] = df_orders["order_id"].astype(str)

mean_orders = df_orders["order_revenue"].mean()

df_orders

##### Segundo Data frame constituido por las órdenes que contenían los productos, cantidad de estos e ingresos

In [ ]:
third_queries = connect.cursor()

third_queries.execute('''select e.EmployeeID, concat (FirstName," ",LastName), sum(order_revenue) as employee_revenue, sum(total_quantity) from Employees e
                        join orders_revenue o on o.EmployeeID = e.EmployeeID 
                        group by e.EmployeeID
                        order by employee_revenue''')

df_employees = pd.DataFrame(third_queries.fetchall())
df_employees.columns = ["employee_id","name","employee_revenue","total_quantity"]

mean_employees = df_employees["employee_revenue"].mean()

df_employees

##### Tercer y último Data Frame constituido por los empleados que realizaron las ventas, cantidad de productos e ingresos

In [130]:
best_product = df_products.iloc[0,:] # Producto con el mayor ingreso de venta
best_product_id = best_product["product_id"]

four_queries = connect.cursor()

four_queries.execute(f'''select OrderID, Quantity from OrderDetails
                        where ProductID = {best_product_id}''')

best_product_quantity_ordered = pd.DataFrame(four_queries.fetchall())
best_product_quantity_ordered.columns = ["order_id","quantity"]

# ---------------------------------------------------------------------------------------------------

best_order = df_orders.iloc[0,:] # Órden con el mayor ingreso de venta
best_order_id = int(best_order["order_id"])

five_queries = connect.cursor()

five_queries.execute(f'''select ProductName, Quantity from Products p
                        join OrderDetails od on od.ProductID = p.ProductID
                        where od.OrderID = {best_order_id}''')

best_order_products = pd.DataFrame(five_queries.fetchall())
best_order_products.columns = ["product", "quantity"]

# ---------------------------------------------------------------------------------------------------

df_employees.sort_values(by="employee_revenue", ascending=False, inplace=True)
best_employee = df_employees.iloc[0,:] # Empleado con el mayor ingreso de venta
best_employee_id = best_employee["employee_id"]

six_queries = connect.cursor()

six_queries.execute(f'''select ProductName, sum(Quantity) from order_details_cash odc
                        join Products p on p.ProductID = odc.ProductID
                        join Orders o on o.OrderID = odC.OrderID join Employees e on o.EmployeeID = e.EmployeeID
                        where e.EmployeeID = {best_employee_id}
                        group by p.ProductID''')

best_employee_products_ordered = pd.DataFrame(six_queries.fetchall())
best_employee_products_ordered.columns = ["product", "quantity"]

In [ ]:
connect.close()

#### Dashboard que refleja los KPIs(Key performance Indicators) y los mejores en desempeño

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(id="body",children=[
    html.Div(id="dropdown_div",className="e3_dropdown_div",children=[
            dcc.Dropdown(id="dropdown",className="e3_dropdown",
                        options = [
                            {"label":"Empleados","value":"name"},
                            {"label":"Productos","value":"product"},
                            {"label":"Órdenes","value":"order_id"}
                        ],
                        value="name",
                        multi=False,
                        clearable=False)
    ]),
    dcc.Graph(id="graph-1",figure={}),
    html.H1("Mayores ingresos",className="e3_title"),
    html.Div(className="e3_container",children=[
        html.Div(id="data_1",className="e3_children",style={"color":"blue"},children=[   
            html.H2("Productos",style={"font-size":"1.15em","color":"blue","font-family":"sans-serif"}),
            html.P(f"Promedio: {round(mean_products,1)}$",className="e3_mean",style={"color":"blue"}),         
            html.Ul(className="e3_ul",style={"color":"blue"},children=[
                html.Li(f"Producto: {best_product["product"]}",className="e3_list"),
                html.Li(f"Cantidad total vendida: {best_product["total_quantity"]}",className="e3_list"),
                html.Li(f"Ingresos totales: {best_product["product_revenue"]}$",className="e3_list")
            ])
        ]),
        html.Div(id="data_2",className="e3_children",children=[
            html.H2("Empleados",style={"font-size":"1.15em","color":"red","font-family":"sans-serif"}),
            html.P(f"Promedio: {round(mean_employees,1)}$",className="e3_mean",style={"color":"red"}),
            html.Ul(className="e3_ul",style={"color":"red"},children=[
                html.Li(f"Nombre: {best_employee["name"]}",className="e3_list"),
                html.Li(f"Cantidad total vendida: {best_employee["total_quantity"]}",className="e3_list"),
                html.Li(f"Ingresos totales: {best_employee["employee_revenue"]}$",className="e3_list")
            ])
        ]),
        html.Div(id="data_3",className="e3_children",children=[
            html.H2("Órdenes",style={"font-size":"1.15em","color":"green","font-family":"sans-serif"}),
            html.P(f"Promedio: {round(mean_orders,1)}$",className="e3_mean",style={"color":"green"}),
            html.Ul(className="e3_ul",style={"color":"green"},children=[
                html.Li(f"Órden: {best_order["order_id"]}",className="e3_list"),
                html.Li(f"Cantidad total vendida: {best_order["total_quantity"]}",className="e3_list"),
                html.Li(f"Ingresos totales: {best_order["order_revenue"]}$",className="e3_list")
            ])
        ]) 
    ]),
    html.H1("Detalles", className="e3_title"),
    dcc.Graph(id="graph-2",figure={})
])

@app.callback(
    [Output(component_id="graph-1",component_property="figure"),
    Output(component_id="graph-2",component_property="figure")],
    [Input(component_id="dropdown",component_property="value")]
)

def update_graph(slct_data):
    
    if slct_data == "name":
        graph_1 = px.bar(df_employees, x=slct_data, y="employee_revenue", color_discrete_sequence=['red'], text_auto='.2s', title="Ingresos de empleados", labels=dict(name="Empleados", employee_revenue="Ingresos"))
        graph_2 = px.bar(best_employee_products_ordered, x="product", y="quantity", color_discrete_sequence=['red'], title="Productos ordenados por el empleado que generó el mayor ingreso", labels=dict(product="Productos", quantity="Cantidad ordenada"))
    elif slct_data == "product":
        graph_1 = px.bar(df_products, x=slct_data, y="product_revenue", color_discrete_sequence=['blue'], text_auto='.2s', title="Ingresos de productos mayores al promedio", labels=dict(product="Productos", product_revenue="Ingresos"))
        graph_1.update_xaxes(tickangle=35, tickfont_size=8)
        graph_2 = px.bar(best_product_quantity_ordered, x="order_id", y="quantity", color_discrete_sequence=['blue'], title="Cantidades ordenadas del producto que generó el mayor ingreso", labels=dict(order_id="Órdenes", quantity="Cantidad ordenada"))
    elif slct_data == "order_id":
        graph_1 = px.bar(df_orders, x=slct_data, y="order_revenue", color_discrete_sequence=['green'], title="Ingresos de órdenes mayores al promedio", labels=dict(order_id="Órdenes", order_revenue="Ingresos"))
        graph_1.update_xaxes(tickfont_size=9)
        graph_2 = px.bar(best_order_products, x="product", y="quantity", color_discrete_sequence=['green'], title="Productos de la órden que generó el mayor ingreso", labels=dict(product="Productos", quantity="Cantidad ordenada"))
    
    return graph_1, graph_2    
    
if __name__ == "__main__":
    app.run_server(debug=False)